In [ ]:
import numpy as np
from math import sqrt
from scipy import stats
from sklearn.metrics import adjusted_rand_score
import pandas as pd


In [ ]:
p_s = [100, 200, 400]
d_s = [2,5,10]
M_s = [4,8,12]
n_percluster_s = [50]
conc_dir_s = [0.25, 0.5, 1]
rho_s = [5, 10, 20]

# Outer cycle for reading the different datasets and perform the estimation
for p,dtrue,M,npc in product(p_s, d_s, M_s, n_percluster_s):
   # Read the latent dimension for the current datasets (already computed since used also in Chandra)
   with open("data/Student_data/latent_dim/stud_p_{0}_d_{1}_M_{2}_npc_{3}_lat_dim.txt".format(p,dtrue,M,npc)) as my_file:
     d = int(my_file.read())

  list_compare = list()

  for conc_dir in conc_dir_s:

    with open("data/Student_data/lamb_out/lamb_p_{0}_d_{1}_M_{2}_npc_{3}_out/conc_{4}_out0/alloc_matrix.csv".format(p,dtrue,M,npc,conc_dir), newline='') as my_csv:
      alloc_matrix = pd.read_csv(my_csv, sep=',', header=None).values

    list_performance = list()
    nclus_lamb = np.array([len(np.unique(alloc_matrix[i,:])) for i in range(alloc_matrix.shape[0])])
    post_mode_nclus_lamb = stats.mode(nclus_lamb)[0][0] # store in dataframe
    post_avg_nclus_lamb = nclus_lamb.mean() # store in dataframe

    best_clus_lamb = cluster_estimate(alloc_matrix)
    true_clus = np.repeat(range(M),npc)
    ari_best_clus_lamb = adjusted_rand_score(true_clus, best_clus_lamb) # store in dataframe

    aris_chain_lamb = np.array([adjusted_rand_score(true_clus, alloc_matrix[i,:]) for i in range(alloc_matrix.shape[0])])
    mean_aris_lamb, sigma_aris_lamb = np.mean(aris_chain_lamb), np.std(aris_chain_lamb) # store mean_aris in dataframe
    CI_aris_lamb = stats.norm.interval(0.95, loc=mean_aris_lamb, scale=sigma_aris_lamb/sqrt(len(aris_chain_lamb))) # store in dataframe
    list_performance.append([p,dtrue,d,M,npc,conc_dir,post_mode_nclus_lamb,
                                post_avg_nclus_lamb,ari_best_clus_lamb,mean_aris_lamb,CI_aris_lamb])
    
    list_compare.append(["lamb",conc_dir, post_mode_nclus_lamb, post_avg_nclus_lamb,ari_best_clus_lamb,mean_aris_lamb,CI_aris_lamb])

    # Save in proper folders similar quantities than for applam
    df_performance_lamb = pd.DataFrame(list_performance, columns=('p','dtrue','d','M','npc','dir_concentr','mode_nclus', 'avg_nclus', 'ari_best_clus', 'CI_aris'))
    df_performance.to_csv("data/Student_data/lamb_out/lamb_p_{0}_d_{1}_M_{2}_npc_{3}_out/conc_{4}_out0/df_performance_lamb.csv".format(p,dtrue,M,npc,conc_dir))
    
    fig = plt.figure()
    plt.plot(nclus_lamb)
    plt.title("number of clusters chain: lamb")
    plt.savefig("data/Student_data/lamb_out/lamb_p_{0}_d_{1}_M_{2}_npc_{3}_out/conc_{4}_out0/nclus_chain_lamb.pdf".format(p,dtrue,M,npc,conc_dir))
    plt.close()


  for rho in rho_s:
    
     with open("data/Student_data/applam/app_p_{0}_d_{1}_M_{2}_npc_{3}_out/rho_{0}_out".format(p,dtrue,M,npc,rho), newline='') as my_csv:
       df_perf_applam = pd.read_csv(my_csv, sep=',')

     list_compare.append(["applam", df_perf_applam[["intensity",'mode_nclus', 'avg_nclus', 'ari_best_clus', 'CI_aris']])


  df_compare = pd.DataFrame(list_compare, columns=('Model','dir_concentr/intens','mode_nclus', 'avg_nclus', 'ari_best_clus', 'CI_aris'))



